<a href="https://colab.research.google.com/github/rupasawant95/chrome-model.py/blob/main/Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Problem Statement :**

**There are times when a user writes Good, Nice App or any other positive text, in the review and gives 1-star rating. Your goal is to identify the reviews where the semantics of review text does not match rating.**

In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

##**Mount google drive**

In [ ]:
#Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Read the dataset
df = pd.read_csv("/content/drive/MyDrive/chrome/chrome_reviews.csv")

In [ ]:
df.head(5)

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [ ]:
df_1=df[['ID','Text','Star']]
df_1.head(3)

,ID,Text,Star
0,3886,This is very helpfull aap.,5
1,3887,Good,3
2,3888,Not able to update. Neither able to uninstall.,1


**Creating another feature called "Result" to classify reviews as positive and negetive based on rating. If rating is >= 2 then it is "positive" represented by 1 else "negative" or not represented by 0.**

In [ ]:
df_1['Result'] = df_1['Star'].apply(lambda x: 1 if x >= 2 else 0)
df_1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Text,Star,Result
0,3886,This is very helpfull aap.,5,1
1,3887,Good,3,1
2,3888,Not able to update. Neither able to uninstall.,1,0
3,3889,Nice app,4,1
4,3890,Many unwanted ads,1,0


##**Data cleaning and processing**

In [ ]:
# data cleaning and preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# creating object for PorterStemmer
ps=PorterStemmer()

# Writing method to clean data
def data_clean_step1(data_set):
  corpus=[]
  for i in range(0,len(data_set)):
    # Removing all the words other than alphabet
    review=re.sub("[^a-zA-Z]"," ",str(data_set['Text'][i]))

    # Converting into lowercase
    review=review.lower()

    #Splitting review as words
    review=review.split()

    # Stemming
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

    # Joining words (making sentences with words stem)
    review=' '.join(review)

    # Making list of reviews
    corpus.append(review)
      
  # replacing processed reviews 
  for i in range(len(corpus)):
    data_set['Text'][i]=corpus[i]

  return data_set

In [ ]:
df_1=data_clean_step1(df_1)
df_1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Text,Star,Result
0,3886,helpful aap,5,1
1,3887,good,3,1
2,3888,abl updat neither abl uninstal,1,0
3,3889,nice app,4,1
4,3890,mani unwant ad,1,0


###**removing the NAN values**

In [ ]:
def data_clean_step2():
  global data_use
  # Removing empty string or ' ' in 'Text' after stemming if any.
  nan_value = float("NaN")
  df_1.replace("", nan_value, inplace=True)
  df_1=df_1.dropna()

  # Resetting the index
  df_1.reset_index(inplace=True)
  corpus=list(df_1['Text'])

In [ ]:
X=df_1['Text'].values
y=df_1['Result'].values

# Splitting the dataset into train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# creating lists to store models, best_scores and best_parameters

models,model_names,best_scores,best_params,test_score=[],[],[],[],[]

###**Creating MultinomialNB model**

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

# Creating MultinomialNB model
from sklearn.naive_bayes import MultinomialNB
text_clf_NB = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfNB', MultinomialNB())])

parameters_NB = {'vect__ngram_range': [(1, 1), (1, 2),(1,3)],'tfidf__use_idf': (True, False),'clfNB__alpha': [0.01,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}

MultinomialNB_classifier = GridSearchCV(text_clf_NB, parameters_NB, n_jobs=-1)
MultinomialNB_classifier = MultinomialNB_classifier.fit(X_train,y_train)

NB_model=MultinomialNB_classifier.best_estimator_
y_pred=NB_model.predict(X_test)

model_names.append('MultinomialNB')
models.append(MultinomialNB_classifier)
best_params.append(MultinomialNB_classifier.best_params_)
best_scores.append(MultinomialNB_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test Accuracy score : ",accuracy_score(y_test,y_pred))
print(MultinomialNB_classifier.best_score_)
print(MultinomialNB_classifier.best_params_)

Test Accuracy score :  0.8501040943789036
0.8394916341428157
{'clfNB__alpha': 0.1, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


In [ ]:
# Creating model for Logisticregression
text_clf_Log = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfLog', LogisticRegression())])
parameters_Log = {'vect__ngram_range': [(1, 1), (1, 2),(1,3)],'tfidf__use_idf': (True, False),
                  'clfLog__penalty':['l1','l2','elasticnet','none'],'clfLog__C':[1.0,1.5,2.0],
                  'clfLog__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                  'clfLog__multi_class':['ovr']}
LogisticRegression_classifier = GridSearchCV(text_clf_Log, parameters_Log, n_jobs=-1)
LogisticRegression_classifier = LogisticRegression_classifier.fit(X_train,y_train)

Logistic_model=LogisticRegression_classifier.best_estimator_
y_pred=Logistic_model.predict(X_test)

model_names.append('LogisticRegression')
models.append(LogisticRegression_classifier)
best_params.append(LogisticRegression_classifier.best_params_)
best_scores.append(LogisticRegression_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred)) 
print(LogisticRegression_classifier.best_score_)
print(LogisticRegression_classifier.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
810 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  

Test accuracy score :  0.8452463566967384
0.8391444119205935
{'clfLog__C': 2.0, 'clfLog__multi_class': 'ovr', 'clfLog__penalty': 'l2', 'clfLog__solver': 'newton-cg', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}


In [ ]:
# Creating model for SGDClassifier
from sklearn.linear_model import SGDClassifier
text_clf_SGD = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfSGD', SGDClassifier())])

parameters_SGD = {'vect__ngram_range': [(1, 1), (1, 2),(1,3),(1,4)],'tfidf__use_idf': (True, False),'clfSGD__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                 'clfSGD__penalty':['l2', 'l1', 'elasticnet'],'clfSGD__alpha':[0.0001,0.001,0.01,0.1,0.2,0.3,0.4,0.5]}

SGD_classifier = GridSearchCV(text_clf_SGD, parameters_SGD, n_jobs=-1)
SGD_classifier = SGD_classifier.fit(X_train,y_train)

SGD_model=SGD_classifier.best_estimator_
y_pred=SGD_model.predict(X_test)

model_names.append('SGDClassifier')
models.append(SGD_classifier)
best_params.append(SGD_classifier.best_params_)
best_scores.append(SGD_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred))
print(SGD_classifier.best_score_)
print(SGD_classifier.best_params_)

Test accuracy score :  0.8445523941707148
0.8398387057916545
{'clfSGD__alpha': 0.0001, 'clfSGD__loss': 'log', 'clfSGD__penalty': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}


In [ ]:
# Creating model for SGDClassifier
from sklearn.svm import SVC
text_clf_SVC = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfSVC', SVC())])

parameters_svc = {'vect__ngram_range': [(1, 1), (1, 2),(1,3),(1,4)],'tfidf__use_idf': (True, False),'clfSVC__C':[0.01,0.1,1,2],
                 'clfSVC__kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],'clfSVC__degree':[1,2,3,4],
                 'clfSVC__gamma':['scale','auto']}

SVC_classifier = GridSearchCV(text_clf_SVC, parameters_svc, n_jobs=-1)
SVC_classifier = SVC_classifier.fit(X_train,y_train)

SVC_model=SVC_classifier.best_estimator_
y_pred=SVC_model.predict(X_test)

model_names.append('SVC_Classifier')
models.append(SVC_classifier)
best_params.append(SVC_classifier.best_params_)
best_scores.append(SVC_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred))
print(SVC_classifier.best_score_)
print(SVC_classifier.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1280 fits failed out of a total of 6400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1280 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 177, in fit
    raise TypeError("Sparse precomputed kernels are not supported.")
TypeError

Test accuracy score :  0.8480222068008327
0.8396638900934759
{'clfSVC__C': 1, 'clfSVC__degree': 1, 'clfSVC__gamma': 'scale', 'clfSVC__kernel': 'poly', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 4)}


In [ ]:
# Creating model for RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

text_clf_RFC = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfRFC', RandomForestClassifier())])

parameters_RFC = {'vect__ngram_range': [(1, 1), (1, 2),(1,3),(1,4)],'tfidf__use_idf': (True, False),
                 'clfRFC__n_estimators':[10,25,50,100],'clfRFC__criterion':['gini','entropy'],
                 'clfRFC__max_depth':[2,4,5,10],'clfRFC__min_samples_leaf':[2,5,10,25,50],
                  'clfRFC__max_features':['auto','sqrt','log2']
                  }

RandomForest_Classifier = GridSearchCV(text_clf_RFC, parameters_RFC, n_jobs=-1)
RandomForest_Classifier = RandomForest_Classifier.fit(X_train,y_train)

RFC_model=RandomForest_Classifier.best_estimator_
y_pred=RFC_model.predict(X_test)

model_names.append('Random_Forest_Classifier')
models.append(RandomForest_Classifier)
best_params.append(RandomForest_Classifier.best_params_)
best_scores.append(RandomForest_Classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred))
print(RandomForest_Classifier.best_score_)
print(RandomForest_Classifier.best_params_)

Test accuracy score :  0.7536433032616239
0.7544736857955093
{'clfRFC__criterion': 'gini', 'clfRFC__max_depth': 10, 'clfRFC__max_features': 'auto', 'clfRFC__min_samples_leaf': 2, 'clfRFC__n_estimators': 10, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}


In [ ]:
for i in range(len(test_score)):
  print(model_names[i])
  print(best_scores[i])
  print(best_params[i])
  print('Test accuracy :',test_score[i])
  print('----------------------------------------------------------------------------------------')

MultinomialNB
0.8394916341428157
{'clfNB__alpha': 0.1, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Test accuracy : 0.8501040943789036
----------------------------------------------------------------------------------------
LogisticRegression
0.8391444119205935
{'clfLog__C': 2.0, 'clfLog__multi_class': 'ovr', 'clfLog__penalty': 'l2', 'clfLog__solver': 'newton-cg', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}
Test accuracy : 0.8452463566967384
----------------------------------------------------------------------------------------
SGDClassifier
0.8396649441071601
{'clfSGD__alpha': 0.0001, 'clfSGD__loss': 'log', 'clfSGD__penalty': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}
Test accuracy : 0.8459403192227619
----------------------------------------------------------------------------------------
SGDClassifier
0.8398387057916545
{'clfSGD__alpha': 0.0001, 'clfSGD__loss': 'log', 'clfSGD__penalty': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}


In [ ]:
import sklearn.externals
import joblib
for i in range(len(model_names)):
  if model_names[i]=='LogisticRegression':
    model=models[i]
joblib.dump(model,'Review_model.mod')

['Review_model.mod']